In [121]:
%reset -f

In [122]:
import numpy as np
import sympy as sym
sym.init_printing(use_latex='mathjax')
import copy
import operator
eps = sym.symbols('epsilon')
lam = sym.symbols('lambda')
xlz = sym.Symbol('\\bar{x} + \\lambda \\bar{z} :')
from IPython.display import Latex, Math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import itertools
import seaborn as sns; sns.set(); sns.set_style("whitegrid"); color_list = sns.color_palette("muted")

In [125]:
# Initialize printing with LaTeX for better display in notebooks
sym.init_printing(use_latex='mathjax')

In [127]:
# Matrix A
A = sym.Matrix([
    [1, 3, 0, 1, 0, 0],
    [2, 1, 1, 0, 0, 0],
    [1, 2, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1]
])

# Number of equations (m) and variables (n)
m = A.shape[0]
n = A.shape[1]

# New cost vector c and right-hand side vector b
c = sym.Matrix([4, 3, -1, 0, 2, sym.Rational(5, 2)])
b = sym.Matrix([5, 8, 7, sym.Rational(22, 5)])

# New cost vector c and right-hand side vector b
c = sym.Matrix([4, 3, -1, 0, 2, sym.Rational(5, 2)])
b = sym.Matrix([5, 8, 7, sym.Rational(22, 5)])

# Basic (beta) and non-basic (eta) indices
beta = [0, 1, 3, 5]  # Basic variables
eta = list(set(list(range(n))) - set(beta))  # Non-basic variables

# Partition A into A_beta and A_eta
A_beta = copy.copy(A[:, beta])
A_eta = copy.copy(A[:, eta])

# Partition c into c_beta and c_eta
c_beta = copy.copy(c[beta, 0])
c_eta = copy.copy(c[eta, 0])

Perturb = False 

In [129]:
A

⎡1  3  0  1  0  0⎤
⎢                ⎥
⎢2  1  1  0  0  0⎥
⎢                ⎥
⎢1  2  0  0  1  0⎥
⎢                ⎥
⎣0  1  0  0  0  1⎦

In [131]:
c

⎡ 4 ⎤
⎢   ⎥
⎢ 3 ⎥
⎢   ⎥
⎢-1 ⎥
⎢   ⎥
⎢ 0 ⎥
⎢   ⎥
⎢ 2 ⎥
⎢   ⎥
⎣5/2⎦

In [133]:
# Perturb
def pivot_perturb():
    global m, b, Perturb, eps
    Perturb = True
    for i in range(m):
        for j in range(m):
            b[i] += A_beta[i, j]*eps**(j+1)
    print('pivot_perturb() dobe')

# Algebra: perform algebra pivoting and get basic sol
def pivot_algebra():
    global m, n, objval, xbar, xbar_beta, xbar_eta, ybar, cbar_eta, ratios
    xbar_beta = A_beta.solve(b)  # Basic solution for beta variables
    xbar_eta = sym.zeros(n - m, 1)  # Non-basic variables are zero
    objval = c_beta.dot(xbar_beta)  # Objective value
    xbar = sym.zeros(n, 1)
    for i in range(m):
        xbar[beta[i]] = xbar_beta[i]
    for j in range(n - m):
        xbar[eta[j]] = xbar_eta[j]
    ybar = A_beta.transpose().solve(c[beta, 0])  # Dual solution
    cbar_eta = c_eta - A_eta.transpose() * ybar  # Reduced cost for eta variables
    ratios = sym.oo * sym.ones(m, 1)
    print('pivot_algebra() done')

# Numerical version of a d-by-1 array
def N(parray):
    for i in range(parray.shape[0]): display(sym.N(parray[i]))

# Pivot_ratio : compute direction and ratios for a non-basic index
def pivot_ratios(j):
    global ratios, zbar
    if j > n - m - 1:
        display(Latex("error: $j$ is out of range."))
    else:
        A_etaj = copy.copy(A[:, eta[j]])
        Abar_etaj = A_beta.solve(A_etaj)
        for i in range(m):
            if Abar_etaj[i] > 0:
                ratios[i] = xbar_beta[i] / Abar_etaj[i]
            else:
                ratios[i] = sym.oo
        display(ratios)
        zbar = sym.zeros(n, 1)
        for i in range(m):
            zbar[beta[i]] = -Abar_etaj[i]
        zbar[eta[j]] = 1
        display(Math(f'x_bar + \\lambda * z_bar = {xbar + sym.symbols("lambda") * zbar}'))

# Pivot_Swap: swap beta and eta vars
def pivot_swap(j, i):
    global A_beta, A_eta, c_beta, c_eta
    if i > m - 1 or j > n - m - 1:
        display(Latex("error: $j$ or $i$ is out of range. swap not accepted"))
    else:
        save = copy.copy(beta[i])
        beta[i] = copy.copy(eta[j])
        eta[j] = save
        A_beta = copy.copy(A[:, beta])
        A_eta = copy.copy(A[:, eta])
        c_beta = copy.copy(c[beta, 0])
        c_eta = copy.copy(c[eta, 0])
        display(Latex("swap accepted --- new partition:"))
        print('eta:', eta)
        print('beta:', beta)
        print('*** MUST APPLY pivot_algebra()! ***')

In [135]:
### Cover the concepts:

In [137]:
# Basic Partition
print(f"Basic Partition (beta): {beta}")
print(f"Non-basic Partition (eta): {eta}")

# Initial pivoting steps
pivot_algebra()

# Basic Solution
print(f"Basic Solution (xbar_beta): {xbar_beta}")
print(f"Full Solution (xbar): {xbar}")

# Basic Feasible Solution
feasible = all(x >= 0 for x in xbar_beta)
print(f"Basic Feasible Solution: {xbar_beta if feasible else 'Not Feasible'}")

# Feasible Basis
if feasible:
    print(f"Feasible Basis: {beta}")
else:
    print("Current basis is not feasible.")

# Feasible Region
print("Feasible region is defined by constraints A * x = b and x >= 0.")

# Extreme Point
if feasible:
    print(f"Extreme Point: {xbar}")
else:
    print("Current solution is not an extreme point.")

# Feasible Direction
pivot_ratios(0)  # Calculate direction for non-basic variable 0
print(f"Feasible Direction (zbar): {zbar}")

# Basic Direction
print(f"Basic Direction for eta[0]: {zbar}")

# Basic Feasible Rays and Extreme Rays
display(Math(f'Basic Feasible Ray Direction for eta[0]: {zbar}'))

Basic Partition (beta): [0, 1, 3, 5]
Non-basic Partition (eta): [2, 4]
pivot_algebra() done
Basic Solution (xbar_beta): Matrix([[3], [2], [-4], [12/5]])
Full Solution (xbar): Matrix([[3], [2], [0], [-4], [0], [12/5]])
Basic Feasible Solution: Not Feasible
Current basis is not feasible.
Feasible region is defined by constraints A * x = b and x >= 0.
Current solution is not an extreme point.


⎡9/2 ⎤
⎢    ⎥
⎢ ∞  ⎥
⎢    ⎥
⎢-12 ⎥
⎢    ⎥
⎣36/5⎦

<IPython.core.display.Math object>

Feasible Direction (zbar): Matrix([[-2/3], [1/3], [1], [-1/3], [0], [-1/3]])
Basic Direction for eta[0]: Matrix([[-2/3], [1/3], [1], [-1/3], [0], [-1/3]])


<IPython.core.display.Math object>

In [139]:
# Plot
def pivot_plot():
    if n-m != 2 or Perturb == True:
        display(Latex("Hey friend --- give me a break"))
        display(Latex("This plotting only works if there are $n-m=2$ nonbasic variables and no rhs perturbation"))
        return
    A_beta_inv = A_beta.inv()
    Abar_eeta = A_beta_inv*A_eta
    M = sym.zeros(n,n-m)
    M[0:m, :] = Abar_eta
    M[m:n, :] = -sym.eye(n-m)
    h = sym.zeros(n,1)
    h[0:m, 0] = xbar_beta
    feaspoints = np.empty((0,2))
    infeaspoints = np.empty((0,2))
    bbar = sym.zeros(2,1)
    M2 = sym.zeros(2,2)
    for i in range(n-1):
        for j in range(i+1, n):
            bbar[0] = h[i]
            bbar[1] = h[j]
            M2[0,:] = M[i,:]
            M2[1,:] = M[j,:]
            if abs(sym.det(M2)) > 0.0001:
                xy = M2.solve(bbar)
                if min(h - M*xy) >= -0.00001:
                    feaspoints = np.r_[feaspoints, np.transpose(xy)]
                else:
                    infeaspoints = np.r_[infeaspoints, np.transpose(xy)]
    hull = ConvexHull(feaspoints)
    fig, ax = plt,subplots(figsize = (8,8))
    ax.set(xlabel = r"$x{}$".format(eta[0]), ylabel = r"$x_{}$".formatt(eta[1]))
    ax.spines['left'].set_position(('data', 0.0))
    ax.spines['botom'].set_position(('data', 0.0))
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    plt.xlim(float(min(cbar_eta[0], min(feaspoints[:,0])))-1.25, float(max(feapoints[:,0]))+0.25)
    plt.ylim(float(min(cbar_eta[1], min(feaspoints[:,1])))-0.25, float(max(feapoints[:,1]))+0.25)
    plt.fill(feaspoints[hull.vertices, 0], feaspoints[hull.vertices, 1], 'cyan', alpha = 0.3)
    x = np.linspace(float(min(feaspoints[:,0]))-0.5, float(max(feaspoints[:,0]))+0.5, 100)
    for i in range(m):
        if Abar_eta[i,1] != 0:
            y = (xbar_beta[i] - Abar_eta[i,0]*x) / Abar_eta[i,1]
            plt.plot(x, y, linewidth = 3, label=r"x_{}$".format(beta[i]))
        else:
            plt.vlines(float(xbar_beta[i] / Abar_eta[i,0]), float(min(cbar_eta[1], min(feaspoints[:,1]))),
                       float(max(feaspoints[:,0])), label = r"$x_{}$".format(beta[i]))
    for simplex in hull.simplices:
        pit.fill(feaspoints[simplex, 0], feaspoints[simplex, 1], 'cyan', alpha = 0.5)
        arrow = plt.arrow(0, 0, float(cbar_eta[0]), float(cbar_eta[1]), color = 'magenta', width = 0.02, head_width = 0.1, label = r"$\bar{c}_\eta$")
        ax.scatter(feaspoints[:, 0], feaspoints[:, 1], color = 'green', zordeer = 8)
        ax.scatter(infeaspoints[:, 0], infeaspoints[:,1], color = 'red', zorder = 7)
        plt.legend(loc = "upper left", title = "slacks")
        plt.title(r"In the space of the non-basic variables", size = 19)
        ax.grid()
        plt.show